In [ ]:
import yaml
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open("demosaic_survey.yml", 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
def md_header(s, size=1):
    return f"{'#'*size} {s}"

def md_bullet(s, indent=0):
    return f"{' '*indent}- {s}"

def md_url(text, link):
    return f"[{text}]({link})"

def md_subsection(d, key, indent=0):
    if key in d:
        return [md_bullet(key,indent)] + [md_bullet(line,indent+4) for line in d[key]]
    else:
        return []
    
def md_table(row_text,col_text,values):
    
    assert((len(row_text), len(col_text)) == values.shape)
    
    def a_row(l):
        return '| ' + '| '.join(l) + '|'
    
    def separator(n_cols):
        return ':-----:'.join(['|' for _ in range(n_cols+1)])
    
    rows = [a_row([' '] + col_text), separator(len(col_text)+1)]
    for i in range(len(row_text)):
        vs = [str(v) for v in values[i,:]]
        rows.append(a_row([row_text[i]] + vs))
        
    return rows

In [ ]:
shorts = [d['short'] for d in data]
shorts_to_idx = {k:i for i,k in enumerate(shorts)}
list(shorts_to_idx.keys()), len(shorts)

In [ ]:
n_papers = len(shorts)
psnrs = [list(d['results'].keys()) for d in data if 'results' in d]
psnrs = sum(psnrs, [])
psnrs = list(set(psnrs))
psnrs = filter(lambda x: x not in ['kodak_sigma15', 'mcm_sigma15'], psnrs)
psnrs = {dataset:np.zeros((n_papers, n_papers)) for dataset in psnrs}
list(psnrs.keys())

In [ ]:
for i, paper in enumerate(shorts):
    d = data[i]
    if 'results' not in d:
        continue
        
    js = list(map(lambda x: shorts_to_idx[x], d['compare_to']))
    
    for k,v in psnrs.items():
        if k in d['results']:
            psnrs[k][i,js] = d['results'][k]
            
            
        
# remove entries with zero col and row
tables = []
    
for k in sorted(psnrs):
    A = psnrs[k]
    nonzero_rows = np.where(A.any(axis=1))[0]
    nonzero_cols = np.where(A.any(axis=0))[0]
    row_text = [shorts[i] for i in range(len(shorts)) if i in nonzero_rows]
    col_text = [shorts[i] for i in range(len(shorts)) if i in nonzero_cols]
    A = A[nonzero_rows,:][:,nonzero_cols]
    
    table = md_table(row_text,col_text,A)
    table = '\n'.join(table)
    table += '\n'*4
    
    tables.append(md_header(k, size=2))
    tables.append('\n')
    tables.append(table)
    

print('\n'.join(tables))

In [ ]:

analysis = [
    md_header('Summary'),
    '\n',
"""
From reading some recent papers on demosaicing. It is obvious to see the following trends

- utilizing neural networks in some capacity in solving the problem. This can range from learning image prior from data in some classical convex optimization frameowork such as ADMM (2017_learned_proximal_operators) to end-to-end training on large datasets for demosaicing [2018_deepdemosaicking, 2019_deepisp]. 
- joint optimization, bundling demosaicing with deblurring, superresolution, denoising [2016_deepjoint, 2017_jointadmm, 2018_jointgan_perceptual, 2018_iterative_resnet_joint], etc. To the extreme, the entire image processing pipeline is under one single framework [2014_flexISP, 2019_deepisp]

I think [2017_learned_proximal_operators, 2017_RED, 2018_iterative_resnet_joint] are most applicable for our project. The idea of these methods is to formulate demosaicing as an inverse problems. The image priors are learnt from data and acts as regularizers in the optimization problem. The benefit of such method is given as follows

- does not require large datasets that deep neural network methods rely on. Usually, small dataset of at most several hundred images is enough to give promising results
- the image priors, i.e. total variation, cross-channel correlation, are learnt from data. 
    - they are not ad hoc, and heuristic based
    - the priors can be learnt for different downstream reconstruction tasks, i.e. multispectral imaging and structured light

There are however problems, for example

- convergence is not guaranteed for nonconvex regularizers.

    - there is some work by [2017_RED] that under some assumptions that most denoising method satisfies, the method is guaranteed to converge.
    - there is some theory used by [this paper](http://openaccess.thecvf.com/content_ICCV_2017/papers/Chang_One_Network_to_ICCV_2017_paper.pdf) which states that under certain assumptions, nonconvex regularizers converges to stationary points

- runtime is slower than interpolation based or purely neural network based methods
 

""",
    '\n'
]


tables_prefix = [
    md_header('Demosaicing Method Performance'),
    '\n',
    """
The following tables keep benchmarked the performance of demosaicing methods over a few datasets

- `kodak`: 24 photos of size 768x512 or 512x768 (http://r0k.us/graphics/kodak/)
- `mcm`: (https://www4.comp.polyu.edu.hk/~cslzhang/CDM_Dataset.htm)
- `msr` (Microsoft dataset): 500 bayer images in both linear RGB and sRGB space (https://www.microsoft.com/en-us/download/details.aspx?id=52535)
- `{kodak,mcm}_sigma20`: gaussian white noise of `\sigma=20` is applied to test images

Row `i` and column `j` of each table indicates the psnr of `j`-th method in `i`-th paper evaluated on a particular dataset
    """
]

In [ ]:
formatted = []
for d in data:
    title = [md_bullet(md_url(d['full'] + f" ({d['short']})", d['url']))]
    notes  = md_subsection(d,'notes',indent=4)
    remark = md_subsection(d,'remark',indent=4)
    
    dformatted = '\n'.join(title+notes+remark)
    dformatted += '\n'*4
    formatted.append(dformatted)

formatted = analysis + tables_prefix + tables + [md_header('Paper summaries'), '\n'] + formatted
with open('demosaic_survey.md', 'w') as f:
    f.write('\n'.join(formatted))